In [ ]:
if __name__ == "__main__":
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    save_path = "./out"
    folds = generate_or_load_folds(X_train, y_train, groups, n_splits=5, save_dir=f"./cv_folds") #groups:Patient ID
    model_list = [
        {"model_type": "ml", "model_name": "logreg", "objective_fn": objective_ml_model, "test_fn": test_ml_model},
        {"model_type": "dl", "model_name": "cnn", "objective_fn": objective_dl_model, "test_fn": test_dl_model}
    ]
    results = {}
    device = torch.device("cuda") 
    for m in model_list:
        print(f"\n🔧 Running model: {m['model_name']} ({m['model_type']})")

        result = run_model_pipeline(
            X_train,
            y_train,
            X_test,
            y_test,
            folds,
            model_type=m["model_type"],
            model_name=m["model_name"],
            objective_fn=m["objective_fn"],
            test_fn=m["test_fn"],
            search_space=None,  # You can also define your own search space for each model
            n_trials=50,
            score_metric="auroc",
            save_path=save_path,
            device=device,
            seed=SEED
        )
        print(f'{m["model_name"]}:{result}')
        results[m["model_name"]] = result

    print("\n📊 Comparison results of all models:")
    for name, res in results.items():
        print(f"{name}: {res}")   